In [ ]:
# | hide

from IPython.display import Markdown as md
from IPython.display import Code


# Cryptocurrency analysis with FastStream

In this tutorial, we will walk through the process of using the `faststream-gen` Python library to generate two `FastStream` applications.

The first application will demonstrate how to retrieve current cryptocurrency prices from various web sources. Subsequently, we will utilize the functionalities of `FastStream` to publish retrieved data as messages to a Kafka topic.


The second application will showcase how to consume messages from the Kafka topic using `FastStream` consumer capabilities. It will process these messages to extract information about cryptocurrencies and calculate the price mean for each cryptocurrency within a certain time window and publish the price mean to another topic.

**Let's get started!**

## Installation

To complete this tutorial, you will need the following software and Python library:

1. Python

2. pip Python package manager

3. faststream-gen

4. A valid OPENAI API key ([click here to get one](https://platform.openai.com/account/api-keys)) 

It is recommended to use a virtual environment for your Python projects. Virtual environments are a common and effective Python development technique that helps to keep dependencies required by different projects separate by creating isolated Python environments for them.

In this tutorial, we will be using Python’s venv module to create a virtual environment.

!!! note

    There are other great third-party tools for creating virtual environments, such as conda and virtualenv, For basic usage, venv is an excellent choice because it already comes packaged with your Python installation. Any of these tools can help you set up a Python virtual environment.

First, create a root directory for this tutorial. Navigate to the desired location and create a new directory called "faststream_gen_tutorial" and enter it.

```sh
mkdir faststream_gen_tutorial
cd faststream_gen_tutorial
```

### Creating and activating a new Python virtual environment

To create a new virtual environment with venv, open a new terminal session in the root directory of your new project and run the command below:

```shl
python3 -m venv venv
```

The above command creates a new virtual environment called venv. Please feel free to change the name if necessary.

Now your project has its own virtual environment. Generally, before you start using it, you’ll first need to activate the environment. Run the below command to activate your new virtual environment:

```sh
source venv/bin/activate
```

### Installing the packages

Before we begin installing our project dependencies, let us first upgrade pip to ensure we are using the most recent packages by running the following command:

```sh
pip install --upgrade pip

```

Now, install the `faststream-gen` package by running the following command:
```sh
pip install faststream-gen
```

If the installation was successful, you should now have faststream-gen installed on your system. To see a full list of settings, run:
```sh
faststream_gen --help
```

Now you have successfully set up the environment and installed the `faststream-gen` package. You are ready to start using it!

## Generate FastStream apps

For generating `FastStream` applications, `faststream-gen` is using `OPENAI` models. So the first step is exporting your `OPENAI_API_KEY`.

```sh
export OPENAI_API_KEY="<your_openai_api_key>"
```

If you don't already have `OPENAI_API_KEY`, you can create one at [OPENAI API keys](https://platform.openai.com/account/api-keys)

### Retrieve and publish app

Now, we will create an application that retrieves information about cryptocurrencies from various web sources and publishes messages to a Kafka topic. In order to achieve this, we need to define the specific locations and formats of the cryptocurrency data we will retrieve, as well as the structure and content of the messages we will produce in the Kafka topic.

Here is the full description of the desired application:

In [ ]:
# | hide

with open('../../docs_src/tutorial/scrape_and_publish_description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [ ]:
# | echo: false

md(description)


```text
Create faststream application which will retrieve cryptocurrency current price and publish it to new_data topic. 
Application should retrieve the data every 2 seconds.

Message which will be produced is JSON with the two attributes:
    - price: non negative float (it represents current pice of cryptocurrency in USD)
    - crypto_currency: string (it represents the cryptocurrency e.g BTC, ETH...)

Curent price of Bitcoin can be retrieveed by simple GET request to 'https://api.coinbase.com/v2/prices/BTC-USD/spot'
Curent price of Ethereum can be retrieveed by simple GET request to 'https://api.coinbase.com/v2/prices/ETH-USD/spot'

The response of this GET request is a JSON and you can get information about the crypto_currency in response['data']['base']
and the information about the price in response['data']['amount']

Use utf-8 encoded crypto_currency attribute as a partition key when publishing the message to new_data topic..
 
```


Let's create a new `FastStream` application and save the application files in a new directory called `retrieve_publish_app`. First, copy the previous description and paste it into a file called `description_retrieve_publish.txt` in the current working directory.

Next, run the following command (parameter `-i` specifies the filepath for the app description file, while the parameter `-o` specifies the directory where the generated project files will be saved.):

```sh
faststream_gen -i description_retrieve_publish.txt -o retrieve_publish_app
```
```console
✨  Generating a new FastStream application!
 ✔ Application description validated. 
 ✔ FastStream app skeleton code generated. 
 ✔ The app and the tests are generated. 
 ✔ New FastStream project created. 
 ✔ Integration tests were successfully completed. 
 Tokens used: 36938
 Total Cost (USD): $0.11436
✨  All files were successfully generated!
```

!!! note

    By default, faststream_gen utilizes `gpt-3.5-turbo` for creating FastStream applications. If you encounter any generation issues, we recommend that you try again with the `--model gpt-4` option. With gpt-4's enhanced capabilities, you can expect even higher success rates, especially when dealing with more complex tasks. 

<br>
This command will generate `retrieve_publish_app` directory with `app/application.py` and `tests/test_application.py` inside.

`app/application.py`:

In [ ]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/scrape_and_publish_app.py', language='python')
md(f"```python\n{code}\n```")

```python
import asyncio
import json

from pydantic import BaseModel, Field, NonNegativeFloat
from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker
import requests

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

publisher = broker.publisher("new_data")


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(
        ..., examples=[50000.0], description="Current price of cryptocurrency in USD"
    )
    crypto_currency: str = Field(
        ..., examples=["BTC"], description="Cryptocurrency symbol e.g BTC, ETH..."
    )


@app.on_startup
async def app_setup(context: ContextRepo):
    context.set_global("app_is_running", True)


@app.on_shutdown
async def app_shutdown(context: ContextRepo):
    context.set_global("app_is_running", False)

    # Get the running task and await for it to finish
    publish_tasks = context.get("publish_tasks")
    await asyncio.wait(publish_tasks)


async def fetch_and_publish_crypto_price(
    crypto_currency: str,
    logger: Logger,
    context: ContextRepo,
    time_interval: int = 2,
) -> None:
    # Always use context: ContextRepo for storing app_is_running variable
    while context.get("app_is_running"):
        if crypto_currency == "BTC":
            url = "https://api.coinbase.com/v2/prices/BTC-USD/spot"
        elif crypto_currency == "ETH":
            url = "https://api.coinbase.com/v2/prices/ETH-USD/spot"
        else:
            logger.warning(f"Invalid crypto currency: {crypto_currency}")
            continue

        response = requests.get(url)

        if response.status_code == 200:
            # read json response
            raw_data = json.loads(response.content)
            price = raw_data["data"]["amount"]

            new_data = CryptoPrice(price=price, crypto_currency=crypto_currency)
            key = crypto_currency.encode("utf-8")
            await publisher.publish(new_data, key=key)
        else:
            logger.warning(f"Failed API request {url}")

        await asyncio.sleep(time_interval)


@app.after_startup
async def publish_crypto_price(logger: Logger, context: ContextRepo):
    logger.info("Starting publishing:")

    crypto_currencies = ["BTC", "ETH"]
    # start fetching and publishing crypto prices
    publish_tasks = [
        asyncio.create_task(
            fetch_and_publish_crypto_price(crypto_currency, logger, context)
        )
        for crypto_currency in crypto_currencies
    ]
    # you need to save asyncio tasks so you can wait for them to finish at app shutdown (the function with @app.on_shutdown function)
    context.set_global("publish_tasks", publish_tasks)

```

### Calculate the moving average app

This application will calculate the mean price of the last three messages received from the 'new_data' topic for each cryptocurrency. Afterwards, the calculated mean price will be published to the 'price_mean' topic.


Here is the full description of the desired application:

In [ ]:
# | hide

with open('../../docs_src/tutorial/calculate_mean_description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [ ]:
# | echo: false

md(description)


```text
Create faststream application for consuming messages from the new_data topic. 
This topic needs to use partition key.

new_data messages use JSON with two attributes (create class CryptoPrice with these attributes)s:
    - price: non negative float (it represents current pice of the crypto)
    - crypto_currency: string (it represents the cryptocurrency e.g BTC, ETH...)

Application should save each message to a dictionary (global variable) - partition key should be usded as a dictionary key and value should be a List of prices.
Keep only last 100 messages in the dictionary.

If there is less the 3 messages for that for the given partition key, do not publish any messages.

Otherwise, Calculate the price mean of the last 3 messages for the given partition key.
Publish the price mean to the price_mean topic and use the same partition key which the new_data topic is using.
 
```


To create a `faststream` application inside the `calculate_mean_app` directory, first, copy the previous description and paste it into the `description_calculate_mean.txt` file.

Next, run the following command:

```sh
faststream_gen -i description_calculate_mean.txt -o calculate_mean_app
```
```console
✨  Generating a new FastStream application!
 ✔ Application description validated. 
 ✔ FastStream app skeleton code generated. 
 ✔ The app and the tests are generated. 
 ✔ New FastStream project created. 
 ✔ Integration tests were successfully completed. 
 Tokens used: 13367
 Total Cost (USD): $0.04147
✨  All files were successfully generated!
```

This command will generate `calculate_mean_app` directory with `app/application.py` and `tests/test_application.py` inside.

`app/application.py`:

In [ ]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/calculate_mean_app.py', language='python')
md(f"```python\n{code}\n```")

```python
from typing import Dict, List

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import Context, ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(
        ..., examples=[50000], description="Current price of the cryptocurrency"
    )
    crypto_currency: str = Field(
        ..., examples=["BTC"], description="Cryptocurrency symbol (e.g. BTC, ETH)"
    )


publisher = broker.publisher("price_mean")


@app.on_startup
async def app_setup(context: ContextRepo):
    message_history: Dict[str, List[float]] = {}
    context.set_global("message_history", message_history)


@broker.subscriber("new_data")
async def on_new_data(
    msg: CryptoPrice,
    logger: Logger,
    message_history: Dict[str, List[float]] = Context(),
    key: bytes = Context("message.raw_message.key"),
) -> None:
    logger.info(f"New data received: {msg=}")

    partition_key = key.decode("utf-8")
    if partition_key not in message_history:
        message_history[partition_key] = []

    message_history[partition_key].append(msg.price)

    if len(message_history[partition_key]) > 100:
        message_history[partition_key].pop(0)

    if len(message_history[partition_key]) >= 3:
        price_mean = sum(message_history[partition_key][-3:]) / 3
        await publisher.publish(price_mean, key=key)

```

## Start localhost Kafka broker


In order for `FastStream` applications to publish and consume messages from the Kafka broker, it is necessary to have a running Kafka broker.

Along with application and test, `faststream-gen` also generated `scripts` directory (inside both app directories).
You can start local Kafka broker by executing following commands:

```sh
cd retrieve_publish_app
# make all shell scripts executable
chmod +x scripts/*.sh
# start local kafka broker
./scripts/start_kafka_broker_locally.sh
```

```console
[+] Running 2/2
 ⠿ Network scripts_default  Created                                                                                                             0.1s
 ⠿ Container bitnami_kafka  Started 
```

## Run the applications

### Retrieve and publish app

To start the `retrieve_publish_app`, navigate to the "faststream_gen_tutorial" directory and run the following command:
```sh
faststream run  retrieve_publish_app.app.application:app
```
```console
2023-09-15 13:41:21,948 INFO     - FastStream app starting...
2023-09-15 13:41:22,144 INFO     -      |            - Starting publishing:
2023-09-15 13:41:22,144 INFO     - FastStream app started successfully! To exit press CTRL+C
Topic new_data not found in cluster metadata
```

### Calculate the moving average app

Open **new terminal**, navigate to the "faststream_gen_tutorial" directory and make sure that you have activated virtual environment.

To start the `calculate_mean_app`, run the following command:
```sh
faststream run calculate_mean_app.app.application:app
```
```console
2023-09-15 13:56:47,245 INFO     - FastStream app starting...
2023-09-15 13:56:47,428 INFO     - new_data |            - `OnNewData` waiting for messages
2023-09-15 13:56:47,621 INFO     - FastStream app started successfully! To exit press CTRL+C
2023-09-15 13:56:48,314 INFO     - new_data | 13675-1694 - Received
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - msg=NewData(price=1624.235, currency='ETH')
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - Processed
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Received
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - msg=NewData(price=26485.545, currency='BTC')
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Processed
2023-09-15 13:56:50,491 INFO     - new_data | 13677-1694 - Received
2023-09-15 13:56:50,492 INFO     - new_data | 13677-1694 - msg=NewData(price=1624.235, currency='ETH')
...
```

You can see in the terminal the that the application is reading the messages from the `price_mean` topic.

### Subscribe directly to local kafka broker topic

Open **new terminal**, navigate to the "faststream_gen_tutorial" directory and make sure that you have activated virtual environment.

To check if the `calculate_mean_app` is publishing messages to the `price_mean` topic, run the following comand:

 ```sh
./retrieve_publish_app/scripts/subscribe_to_kafka_broker_locally.sh price_mean
```
```console
BTC     26405.745
ETH     1621.3733333333332
BTC     26404.865
ETH     1621.375
BTC     26404.865
...
```

To stop the Kafka broker after analyzing the mean price of cryptocurrencies, you can execute the following commands:
```sh
cd retrieve_publish_app 
./scripts/stop_kafka_broker_locally.sh
```
```console
[+] Running 2/2
 ⠿ Container bitnami_kafka  Removed                                                                            1.2s
 ⠿ Network scripts_default  Removed 
```

## Next steps

Congratulations! You have successfully completed this tutorial and gained a new set of skills. Now that you have learned how to use `faststream-gen`, try it out with your own example!